<a href="https://colab.research.google.com/github/momenashra/Obects/blob/main/Objects_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!pip install transformers gradio

# Import Dependencies

In [2]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

# First Model (Coder)

In [3]:
class DeepSeekCoderApp:
    def __init__(self):
        self.model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = None
        self.tokenizer = None
        self.processor = None

    def load_model(self):
        """Load the DeepSeek model and tokenizer"""
        print(f"Loading model on {self.device}...")
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            self.model_name,
            trust_remote_code=True,
            padding_side="left"
        )
        # Load model with appropriate settings for T4 GPU
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            device_map="auto" ,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        # Set pad token if not present
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        print("Model loaded successfully!")

    def generate_text(self, prompt, temperature=0.3, top_p=0.9, max_new_tokens=512):
        """Generate text based on prompt with adjustable parameters"""
        if self.model is None:
            return "Model not loaded. Please wait for initialization."
        try:
            # Tokenize input
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=2048
            )
            # Move each tensor to the right device
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=temperature,
                    top_p=top_p,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )
            # Decode response
            response = self.tokenizer.decode(
                outputs[0][inputs['input_ids'].shape[1]:],
                skip_special_tokens=True
            )
            return response.strip()
        except Exception as e:
            return f"Error generating text: {str(e)}"

    def generate_code(self, prompt, temperature=0.1, top_p=0.8, max_new_tokens=2000):
        """Generate code based on natural language prompt"""
        if not prompt.strip():
            return "Please provide a code generation prompt."
                    # Create structured prompt for code generation
        code_prompt = f"""<|im_start|>user
                        {prompt.strip()} in Python

                        Requirements:
                        1. Start with the code immediately
                        2. Complete, working implementation
                        3. Essential comments only
                        4. Proper error handling
                        5. Clean formatting
                        6. No explanations outside the code
                        7. Use best practices
                        8. Include docstrings for functions

                        <|im_end|>
                        <|im_start|>assistant
                        ```python"""
        response = self.generate_text(code_prompt, temperature, top_p, max_new_tokens)
        # Clean up the response and ensure proper code formatting
        if not response.startswith("```"):
            response = "```python\n" + response
        if not response.endswith("```"):
            response = response + "\n```"
        return response

In [4]:
coder = DeepSeekCoderApp()
coder.load_model()

Loading model on cpu...


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model loaded successfully!


# Scond Model (Image Captioner)

In [5]:
class LlavaApp:
    def __init__(self):
        self.model_id = "llava-hf/llava-1.5-7b-hf"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = None
        self.processor = None

    def load_model(self):
        """Load the Llava model and processor - FIXED VERSION"""
        print(f"Loading model on {self.device}...")

        # Load processor
        self.processor = AutoProcessor.from_pretrained(self.model_id)

        # FIXED: Load model with device_map="auto" and DON'T call .to("cuda")
        self.model = LlavaForConditionalGeneration.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            device_map="auto"  # This handles device placement automatically

        )

        print("Model loaded successfully!")

    def generate_text(self, prompt, image,temperature,top_p,max_tokens):
        """Generate text based on prompt and image"""
        code_prompt = f"""<|im_start|>user
                          {prompt}
                          Requirements:
                          - Detailed explaination about you see in the image
                          - identify objects you can see

                          <|im_end|>
                          <|im_start|>assistant
                          ```Answer"""
        if not prompt.strip():
            return "Please provide a prompt."
        # Prompt must include <image> token
        if "<image>" not in prompt:
            prompt = f"{prompt} <image>"
        # Preprocess text + image together using processor
        inputs = self.processor(text=prompt, images=image, return_tensors="pt").to(self.model.device)
        # Generate response
        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=top_p
            )
        # Decode response
        response = self.processor.tokenizer.decode(output[0], skip_special_tokens=True)
        return response

In [6]:
describer=LlavaApp()
describer.load_model()

Loading model on cpu...


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Model loaded successfully!


## Qucik Testing Function

In [7]:
def process_input(image, prompt, temperature, top_p, max_tokens):
    if image is not None:
        image = image.convert("RGB")
        # Ensure <image> token is in the prompt
        if "<image>" not in prompt:
            prompt = f"{prompt} <image>"
        response = describer.generate_text(
            prompt,
            image,
            temperature,
            top_p,
            max_tokens
        )
        print(f"\nPrompt: {prompt[:100]}...")
        print(f"Response: {response}")
        return response
    else:
        """Quick single test function"""
        response = coder.generate_code(
            prompt,
            temperature=temperature,
            top_p=top_p,
            max_new_tokens=max_tokens
        )
        print(f"\nPrompt: {prompt[:100]}...")
        print(f"Response: {response}")
        return response

In [8]:
# Create Gradio interface
iface = gr.Interface(
    fn=process_input,
    inputs=[
        gr.Image(type="pil", label="Upload Image (Optional)"),
        gr.Textbox(label="Prompt", placeholder="Enter your prompt (include <image> for image description, or describe code to generate)"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.9, step=0.05, label="Top P"),
        gr.Slider(minimum=50, maximum=550, value=300, step=20, label="Max Tokens")
    ],
    outputs=gr.Textbox(label="Output"),
    title="Dual-Function Image Description and Code Generation",
    description="Upload an image to get its description using Llava, or provide a code-related prompt to generate code using DeepSeek . Adjust temperature, top_p, and max tokens to control output creativity and length.",
    examples=[
        [None, "Fully describe the provided image", 0.7, 0.9, 100],  # Example for image description adding high creativity
        [None, "Write a Python function to calculate the factorial of a number.", 0.1, 0.6,300 ]  # Example for code generation adding high accuracy
    ]
)

In [9]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee035aec64804fdbb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
